In [1]:
import json

# Load the configuration from the JSON file
with open('D:/BrainAge/HCCT/config.json', 'r') as f:
    config = json.load(f)

In [2]:
from model import ViTForClassfication

# Initialize the model with the loaded configuration
model = ViTForClassfication(config=config)

In [5]:
import torch
extra_features= torch.randn(1,108)
x = torch.randn(1, 1, 91, 109, 91)
# output= model(x, extra_features)
output = model(x, output_attentions=True, return_cam=False)

In [6]:
len(output)

2

In [15]:
a,b,c,d= output

In [16]:
# Age
a[0].item()

-0.06654798239469528

In [ ]:
# last layer feature map
b.shape

torch.Size([1, 512, 2, 3, 2])

In [ ]:
# last layer grad
c.shape

torch.Size([1, 512, 2, 3, 2])

In [ ]:
# Attention probabilities
len(d)
d[0].shape

torch.Size([1, 8, 513, 513])